<font size="18">Image Augmentation App</font>

This application demonstrates a simple rocAL pipeline with different interpolation types for resize augmentation supported by rocAL.

<font size="12"> Common Code </font>

In [ ]:
from amd.rocal.plugin.generic import ROCALClassificationIterator
from amd.rocal.pipeline import Pipeline
import amd.rocal.fn as fn
import amd.rocal.types as types
import matplotlib.pyplot as plt
%matplotlib inline

<font size= "12" >Configuring rocAL pipeline </font>

Configure the pipeline parameters as required by the user.

In [ ]:
data_path = "/media/MIVisionX-data/rocal_data/coco/coco_10_img/train_10images_2017/"
rocal_cpu = True
device = "cpu"
batch_size = 1
num_threads = 1
random_seed = 1
local_rank = 0
world_size = 1
display = True

In [ ]:
pipe = Pipeline(batch_size=batch_size, num_threads=num_threads, device_id=local_rank, seed=random_seed, rocal_cpu=rocal_cpu, tensor_layout=types.NHWC, tensor_dtype=types.FLOAT)


<font size="12"> Image augmentation pipeline </font>

Here the file reader is used followed by the turbo jpeg decoder. In this pipeline, cascaded augmentations are added on the decoded images.<br>Multiple augmentation outputs are returned using set_outputs

In [ ]:
resize_w = 200
resize_h = 200
with pipe:
        jpegs, _ = fn.readers.file(file_root=data_path)
        images = fn.decoders.image(jpegs, file_root=data_path, device=0, output_type=types.RGB, shard_id=0, num_shards=1, random_shuffle=False)
        output_linear = fn.resize(images, resize_width=resize_w, resize_height=resize_h, interpolation_type=types.LINEAR_INTERPOLATION)
        output_nearest_neighbor = fn.resize(images, resize_width=resize_w, resize_height=resize_h, interpolation_type=types.NEAREST_NEIGHBOR_INTERPOLATION)
        output_cubic = fn.resize(images, resize_width=resize_w, resize_height=resize_h, interpolation_type=types.CUBIC_INTERPOLATION)
        output_lanczos = fn.resize(images, resize_width=resize_w, resize_height=resize_h, interpolation_type=types.LANCZOS_INTERPOLATION)
        output_gaussian = fn.resize(images, resize_width=resize_w, resize_height=resize_h, interpolation_type=types.GAUSSIAN_INTERPOLATION)
        output_triangular = fn.resize(images, resize_width=resize_w, resize_height=resize_h, interpolation_type=types.TRIANGULAR_INTERPOLATION)
        pipe.set_outputs(output_linear, output_nearest_neighbor, output_cubic, output_lanczos, output_gaussian, output_triangular)


In [ ]:
pipe.build()
# Dataloader
data_loader = ROCALClassificationIterator(pipe)

<font size ="12">Visualizing  outputs</font>

The output of augmented images are displayed using imshow()

In [ ]:
cnt = 0
aug_list = ["LINEAR_INTERPOLATION", "NEAREST_NEIGHBOR_INTERPOLATION", "CUBIC_INTERPOLATION", "LANCZOS_INTERPOLATION", "GAUSSIAN_INTERPOLATION", "TRIANGULAR_INTERPOLATION"] 
row = 0
col = 0
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 15))
for i, it in enumerate(data_loader, 0):
    for img in it[0]:
        if cnt < len(aug_list):
            axes[row, col].set_title(aug_list[cnt])
        img = (img[0]).astype("uint8")
        axes[row, col].imshow(img)
        cnt += 1
        row += 1
        if row == 3:
            row = 0
            col += 1
        if col == 2:
            col = 0
data_loader.reset()